# Gaussian Process Regression

## import libs

In [2]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

import pandas as pd
import numpy as np
from scipy import stats
from matplotlib import pyplot as plt

import time

## read data

In [3]:
## grid sampling 1296
data1296 = pd.read_csv('CLEANED_gridsearch_1296.csv')
data1296 = data1296.drop(data1296.columns[0], axis=1)
X_1296 = data1296.drop('density', axis=1)
Y_1296 = data1296['density']
#print(f'{data1296}')
#print(f'{X_1296}')
#print(f'{Y_1296}')

## grid sampling 2401
data2401 = pd.read_csv('CLEANED_gridsearch_2401.csv')
data2401 = data2401.drop(data2401.columns[0], axis=1)
X_2401 = data2401.drop('density', axis=1)
Y_2401 = data2401['density']
#print(f'{data2401}')
#print(f'{X_2401}')
#print(f'{Y_2401}')

## sobol2 sampling
data_sobol1 = pd.read_csv('CLEANED_sobolsampling-2048.csv')
data_sobol1 = data_sobol1.drop(data_sobol1.columns[0], axis=1)
X_sobol1 = data_sobol1.drop('density', axis=1)
Y_sobol1 = data_sobol1['density']
#print(f'{data_sobol1}')
#print(f'{X_sobol1}')
#print(f'{Y_sobol1}')

## sobol2 sampling
data_sobol2 = pd.read_csv('CLEANED_sobolsampling-2048-2.csv')
data_sobol2 = data_sobol2.drop(data_sobol2.columns[0], axis=1)
X_sobol2 = data_sobol2.drop('density', axis=1)
Y_sobol2 = data_sobol2['density']
#print(f'{data_sobol2}')
#print(f'{X_sobol2}')
#print(f'{Y_sobol2}')

## prepare Data

In [4]:
## suggestions for random ints
#random_array = np.random.rand(30)*100
#random_array = random_array.astype(int)
#print(f'{random_array}')

## actually used random ints hard coded
random_ints = [678, 147, 561, 237, 588, 951, 490, 395, 877, 297, 721, 711, 985, 171, 75, 16, 669, 530, 999, 794, 936, 111, 816, 968, 48, 986, 829, 996, 272, 759, 390, 930, 633, 928, 854, 554, 562, 78, 222, 294, 725, 582, 731, 249, 791, 35, 180, 510, 593, 634]
#print(f'{np.sort(random_ints)}')
trainingsizes = [0.05, 0.10, 0.15, 0.20, 0.25, 0.30, 0.35, 0.40, 0.45, 0.50, 0.55, 0.60, 0.65, 0.70, 0.75, 0.80, 0.85, 0.90, 0.95]

## split datasets
create train/test sets based on random_ints and trainingsizes

In [5]:
## grid sampling 1296
X_TRAINs1296 = []
X_TESTs1296 = []
Y_TRAINs1296 = []
Y_TESTs1296 = []

## grid sampling 2401
X_TRAINs2401 = []
X_TESTs2401 = []
Y_TRAINs2401 = []
Y_TESTs2401 = []

## sobol sampling 1
X_TRAINsSobol1 = []
X_TESTsSobol1 = []
Y_TRAINsSobol1 = []
Y_TESTsSobol1 = []

## sobol sampling 2
X_TRAINsSobol2 = []
X_TESTsSobol2 = []
Y_TRAINsSobol2 = []
Y_TESTsSobol2 = []

for t in trainingsizes:
    testsize = np.round(1.0 - np.round(float(t), 2), 2)
    print(f'testsize = {testsize}')
    ## grid sampling 1296
    qX_TRAINs1296 = []
    qX_TESTs1296 = []
    qY_TRAINs1296 = []
    qY_TESTs1296 = []
    
    ## grid sampling 2401
    qX_TRAINs2401 = []
    qX_TESTs2401 = []
    qY_TRAINs2401 = []
    qY_TESTs2401 = []
    
    ## sobol sampling 1
    qX_TRAINsSobol1 = []
    qX_TESTsSobol1 = []
    qY_TRAINsSobol1 = []
    qY_TESTsSobol1 = []
    
    ## sobol sampling 2
    qX_TRAINsSobol2 = []
    qX_TESTsSobol2 = []
    qY_TRAINsSobol2 = []
    qY_TESTsSobol2 = []

    
    for i in random_ints:
        #print(f'{i}')
        ## use the X_test, Y_test data for testing combined with all the data of the other datasets
        X_train, X_test, Y_train, Y_test = train_test_split(X_1296, Y_1296, test_size=testsize, random_state=i)
        qX_TRAINs1296.append(X_train)
        qY_TRAINs1296.append(Y_train)
        #print(f'{X_test}')
        #print(f'{Y_test}')
        X_test = pd.concat([X_test, X_2401, X_sobol1, X_sobol2], ignore_index=True)
        Y_test = pd.concat([Y_test, Y_2401, Y_sobol1, Y_sobol2], ignore_index=True)
        qX_TESTs1296.append(X_test)
        qY_TESTs1296.append(Y_test)
        #print(f'{X_test}')
        #print(f'{Y_test}')
        
            
        X_train, X_test, Y_train, Y_test = train_test_split(X_2401, Y_2401, test_size=testsize, random_state=i)
        qX_TRAINs2401.append(X_train),
        qY_TRAINs2401.append(Y_train)
        X_test = pd.concat([X_test, X_1296, X_sobol1, X_sobol2], ignore_index=True)
        Y_test = pd.concat([Y_test, Y_1296, Y_sobol1, Y_sobol2], ignore_index=True)
        qX_TESTs2401.append(X_test)
        qY_TESTs2401.append(Y_test)
        
            
        X_train, X_test, Y_train, Y_test = train_test_split(X_sobol1, Y_sobol1, test_size=testsize, random_state=i)
        qX_TRAINsSobol1.append(X_train)
        qY_TRAINsSobol1.append(Y_train)
        X_test = pd.concat([X_test, X_1296, X_2401, X_sobol2], ignore_index=True)
        Y_test = pd.concat([Y_test, Y_1296, Y_2401, Y_sobol2], ignore_index=True)
        qX_TESTsSobol1.append(X_test)
        qY_TESTsSobol1.append(Y_test)
        
            
        X_train, X_test, Y_train, Y_test = train_test_split(X_sobol2, Y_sobol2, test_size=testsize, random_state=i)
        qX_TRAINsSobol2.append(X_train)
        qY_TRAINsSobol2.append(Y_train)
        X_test = pd.concat([X_test, X_1296, X_2401, X_sobol1], ignore_index=True)
        Y_test = pd.concat([Y_test, Y_1296, Y_2401, Y_sobol1], ignore_index=True)
        qX_TESTsSobol2.append(X_test)
        qY_TESTsSobol2.append(Y_test)

    ## grid sampling 1296
    X_TRAINs1296.append(qX_TRAINs1296)
    X_TESTs1296.append(qX_TESTs1296)
    Y_TRAINs1296.append(qY_TRAINs1296)
    Y_TESTs1296.append(qY_TESTs1296)
    
    ## grid sampling 2401
    X_TRAINs2401.append(qX_TRAINs2401)
    X_TESTs2401.append(qX_TESTs2401)
    Y_TRAINs2401.append(qY_TRAINs2401)
    Y_TESTs2401.append(qY_TESTs2401)
    
    ## sobol sampling 1
    X_TRAINsSobol1.append(qX_TRAINsSobol1)
    X_TESTsSobol1.append(qX_TESTsSobol1)
    Y_TRAINsSobol1.append(qY_TRAINsSobol1)
    Y_TESTsSobol1.append(qY_TESTsSobol1)
    
    ## sobol sampling 2
    X_TRAINsSobol2.append(qX_TRAINsSobol2)
    X_TESTsSobol2.append(qX_TESTsSobol2)
    Y_TRAINsSobol2.append(qY_TRAINsSobol2)
    Y_TESTsSobol2.append(qY_TESTsSobol2)

testsize = 0.95
testsize = 0.9
testsize = 0.85
testsize = 0.8
testsize = 0.75
testsize = 0.7
testsize = 0.65
testsize = 0.6
testsize = 0.55
testsize = 0.5
testsize = 0.45
testsize = 0.4
testsize = 0.35
testsize = 0.3
testsize = 0.25
testsize = 0.2
testsize = 0.15
testsize = 0.1
testsize = 0.05


## create and train models

In [6]:
## grid sampling 1296
MODELs1296 = []
Y_PREDICTIONs1296_MEAN = []
Y_PREDICTIONs1296_STD = []
RMSEs1296 = []
R2s1296 = []
SPEARMANRs1296 = []

## grid sampling 2401
MODELs2401 = []
Y_PREDICTIONs2401_MEAN = []
Y_PREDICTIONs2401_STD = []
RMSEs2401 = []
R2s2401 = []
SPEARMANRs2401 = []

## sobol sampling 1
MODELsSobol1 = []
Y_PREDICTIONsSobol1_MEAN = []
Y_PREDICTIONsSobol1_STD = []
RMSEsSobol1 = []
R2sSobol1 = []
SPEARMANRsSobol1 = []

## sobol sampling 2
MODELsSobol2 = []
Y_PREDICTIONsSobol2_MEAN = []
Y_PREDICTIONsSobol2_STD = []
RMSEsSobol2 = []
R2sSobol2 = []
SPEARMANRsSobol2 = []

kernel = 1 * RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e2))
n_restarts = 9

for k in range(0, len(trainingsizes)):
    print(f'Training model with {trainingsizes[k]}% of the Data for training and {1-trainingsizes[k]}% for testing ...')
    ## grid sampling 1296
    qMODELs1296 = []
    qY_PREDICTIONs1296_mean = []
    qY_PREDICTIONs1296_std = []
    qRMSEs1296 = []
    qR2s1296 = []
    qSPEARMANRs1296 = []
    
    ## grid sampling 2401
    qMODELs2401 = []
    qY_PREDICTIONs2401_mean = []
    qY_PREDICTIONs2401_std = []
    qRMSEs2401 = []
    qR2s2401 = []
    qSPEARMANRs2401 = []
    
    ## sobol sampling 1
    qMODELsSobol1 = []
    qY_PREDICTIONsSobol1_mean = []
    qY_PREDICTIONsSobol1_std = []
    qRMSEsSobol1 = []
    qR2sSobol1 = []
    qSPEARMANRsSobol1 = []
    
    ## sobol sampling 2
    qMODELsSobol2 = []
    qY_PREDICTIONsSobol2_mean = []
    qY_PREDICTIONsSobol2_std = []
    qRMSEsSobol2 = []
    qR2sSobol2 = []
    qSPEARMANRsSobol2 = []
    
    for i in range(0, len(random_ints)):
        print(f'\t{i+1}/{len(random_ints)}')
        ## create the model
        model1296 = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=n_restarts, random_state=random_ints[i])
        #
        model2401 = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=n_restarts, random_state=random_ints[i])
        #
        modelSobol1 = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=n_restarts, random_state=random_ints[i])
        #
        modelSobol2 = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=n_restarts, random_state=random_ints[i])
        
        ## train/fit the model
        model1296.fit(X_TRAINs1296[k][i], Y_TRAINs1296[k][i])
#        qMODELs1296.append(model1296)
        #
        model2401.fit(X_TRAINs2401[k][i], Y_TRAINs2401[k][i])
#        qMODELs2401.append(model2401)
        #
        modelSobol1.fit(X_TRAINsSobol1[k][i], Y_TRAINsSobol1[k][i])
#        qMODELsSobol1.append(modelSobol1)
        #
        modelSobol2.fit(X_TRAINsSobol2[k][i], Y_TRAINsSobol2[k][i])
#        qMODELsSobol2.append(modelSobol2)
        
        ## prediction using the test set
        Y_prediction1296_mean, Y_prediction1296_std = model1296.predict(X_TESTs1296[k][i], return_std=True)
        qY_PREDICTIONs1296_mean.append(Y_prediction1296_mean)
        qY_PREDICTIONs1296_std.append(Y_prediction1296_std)
        #
        Y_prediction2401_mean, Y_prediction2401_std = model2401.predict(X_TESTs2401[k][i], return_std=True)
        qY_PREDICTIONs2401_mean.append(Y_prediction2401_mean)
        qY_PREDICTIONs2401_std.append(Y_prediction2401_std)
        #
        Y_predictionSobol1_mean, Y_predictionSobol1_std = modelSobol1.predict(X_TESTsSobol1[k][i], return_std=True)
        qY_PREDICTIONsSobol1_mean.append(Y_predictionSobol1_mean)
        qY_PREDICTIONsSobol1_std.append(Y_predictionSobol1_std)
        #
        Y_predictionSobol2_mean, Y_predictionSobol2_std = modelSobol2.predict(X_TESTsSobol2[k][i], return_std=True)
        qY_PREDICTIONsSobol2_mean.append(Y_predictionSobol2_mean)
        qY_PREDICTIONsSobol2_std.append(Y_predictionSobol2_std)
        
        ## evaluate with Y_test
        rmse1296 = np.sqrt(mean_squared_error(Y_TESTs1296[k][i], Y_prediction1296_mean))
        r21296 = r2_score(Y_TESTs1296[k][i], Y_prediction1296_mean)
        spearman_r1296 = stats.spearmanr(Y_TESTs1296[k][i], Y_prediction1296_mean)
        qRMSEs1296.append(rmse1296)
        qR2s1296.append(r21296)
        qSPEARMANRs1296.append(spearman_r1296.statistic)
        #
        rmse2401 = np.sqrt(mean_squared_error(Y_TESTs2401[k][i], Y_prediction2401_mean))
        r22401 = r2_score(Y_TESTs2401[k][i], Y_prediction2401_mean)
        spearman_r2401 = stats.spearmanr(Y_TESTs2401[k][i], Y_prediction2401_mean)
        qRMSEs2401.append(rmse2401)
        qR2s2401.append(r22401)
        qSPEARMANRs2401.append(spearman_r2401.statistic)
        #
        rmseSobol1 = np.sqrt(mean_squared_error(Y_TESTsSobol1[k][i], Y_predictionSobol1_mean))
        r2Sobol1 = r2_score(Y_TESTsSobol1[k][i], Y_predictionSobol1_mean)
        spearman_rSobol1 = stats.spearmanr(Y_TESTsSobol1[k][i], Y_predictionSobol1_mean)
        qRMSEsSobol1.append(rmseSobol1)
        qR2sSobol1.append(r2Sobol1)
        qSPEARMANRsSobol1.append(spearman_rSobol1.statistic)
        #
        rmseSobol2 = np.sqrt(mean_squared_error(Y_TESTsSobol2[k][i], Y_predictionSobol2_mean))
        r2Sobol2 = r2_score(Y_TESTsSobol2[k][i], Y_predictionSobol2_mean)
        spearman_rSobol2 = stats.spearmanr(Y_TESTsSobol2[k][i], Y_predictionSobol2_mean)
        qRMSEsSobol2.append(rmseSobol2)
        qR2sSobol2.append(r2Sobol2)
        qSPEARMANRsSobol2.append(spearman_rSobol2.statistic)

    ## grid sampling 1296
#    MODELs1296.append(qMODELs1296)
    Y_PREDICTIONs1296_MEAN.append(qY_PREDICTIONs1296_mean)
    Y_PREDICTIONs1296_STD.append(qY_PREDICTIONs1296_std)
    RMSEs1296.append(qRMSEs1296)
    R2s1296.append(qR2s1296)
    SPEARMANRs1296.append(qSPEARMANRs1296)
    
    ## grid sampling 2401
#    MODELs2401.append(qMODELs2401)
    Y_PREDICTIONs2401_MEAN.append(qY_PREDICTIONs2401_mean)
    Y_PREDICTIONs2401_STD.append(qY_PREDICTIONs2401_std)
    RMSEs2401.append(qRMSEs2401)
    R2s2401.append(qR2s2401)
    SPEARMANRs2401.append(qSPEARMANRs2401)
    
    ## sobol sampling 1
#    MODELsSobol1.append(qMODELsSobol1)
    Y_PREDICTIONsSobol1_MEAN.append(qY_PREDICTIONsSobol1_mean)
    Y_PREDICTIONsSobol1_STD.append(qY_PREDICTIONsSobol1_std)
    RMSEsSobol1.append(qRMSEsSobol1)
    R2sSobol1.append(qR2sSobol1)
    SPEARMANRsSobol1.append(qSPEARMANRsSobol1)
    
    ## sobol sampling 2
#    MODELsSobol2.append(qMODELsSobol2)
    Y_PREDICTIONsSobol2_MEAN.append(qY_PREDICTIONsSobol2_mean)
    Y_PREDICTIONsSobol2_STD.append(qY_PREDICTIONsSobol2_std)
    RMSEsSobol2.append(qRMSEsSobol2)
    R2sSobol2.append(qR2sSobol2)
    SPEARMANRsSobol2.append(qSPEARMANRsSobol2)

print(f'done.\n')

Training model with 0.05% of the Data for training and 0.95% for testing ...
	1/50


C:\Users\H-BRS\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\gaussian_process\kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\H-BRS\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\gaussian_process\kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\H-BRS\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\gaussian_process\_gpr.py:660: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-lear

	2/50


C:\Users\H-BRS\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\gaussian_process\_gpr.py:660: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
C:\Users\H-BRS\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\gaussian_process\kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\H-BRS\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\gaussian_process\kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. In

KeyboardInterrupt: 